In [31]:
import tensorflow as tf
import glob
import os
import numpy as np

In [32]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def convolution_graph(param_lambda=0.01):
    x = tf.placeholder(tf.float32, shape=[None, 784])
    y_ = tf.placeholder(tf.float32, shape=[None, 10])


    x_input= tf.reshape(x, [-1, 28, 28, 1])

    W_conv1 = weight_variable([5, 5, 1, 16])
    b_conv1 = bias_variable([16])

    h_conv1 = tf.nn.relu(conv2d(x_input, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    print (h_pool1.shape)

    W_conv2 = weight_variable([5, 5, 16, 32])
    b_conv2 = bias_variable([32])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([7 * 7 * 32, 512])
    b_fc1 = bias_variable([512])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([512, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    l2=sum([param_lambda*tf.nn.l2_loss(x) for x in [W_conv2,W_conv1,W_fc2,W_fc1]])

    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))+l2
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
def train():
    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      for i in range(300):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
          train_accuracy = accuracy.eval(feed_dict={
              x: batch[0], y_: batch[1], keep_prob: 1.0})
          print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

      #print('test accuracy %g' % accuracy.eval(feed_dict={
    #      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [57]:
#data preparation

home=os.getenv("HOME")
source="/C_bytecode_analysis/output"
data=[]
number_of_lines=0
offset=0
data_label_dict={}
for x in ["FOR","CALL","IF","ELSE","WHILE","DO","SWITCH","FUNCTION"]:
    data_label_dict[x]=[]

for filename in glob.glob(os.path.join(home+source,'*.labeled_addresses')):
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            line=line.strip().split("\t")
            if len(line)==3:
                func_name=line[0]
                classes=line[1]
                number_of_lines=int(line[2])
                label_list=[]
                offset=len(data)
                #print (func_name, classes, number_of_lines)
            else:
                #address = line[0]
                bytecode = line[1].strip()
                #instruction = line[2].strip()
                #source_line = int(line[3])
                #source_file = line[4]
                labels = line[5:]
                label_list.append(labels)
                #print (address,bytecode,instruction,source_line,source_file,labels)
                indexes= (str(bytecode))[2:4]
                if len(bytecode)==4:
                    indexes+="gg"
                else:
                    indexes+=(str(bytecode))[4:6]
                new=[]
                for x in indexes:
                        pom=[0]*16
                        if x!="g":
                            pom[int(x,16)]=1
                        new.extend(pom)
                data.append(np.array(new))
                number_of_lines-=1
                if number_of_lines==0:
                    cur=[]
                    counter={}
                    start_dict={}
                    i=-1
                    for labels in label_list:
                        i+=1
                        for label in labels:
                            if label not in cur:
                                cur.append(label)
                                counter[label]=1
                                start_dict[label]=i
                            else:
                                counter[label]+=1
                    for label in counter.keys():
                        data_label_dict[label.split("_")[0]].append((start_dict[label]+offset,counter[label]))
                                
        print(data_label_dict)

{'FOR': [(4, 8), (32, 11), (34, 6), (50, 26)], 'CALL': [(12, 5), (27, 5), (45, 5), (52, 5), (76, 1), (83, 3), (86, 2), (93, 5)], 'IF': [(57, 6), (77, 9)], 'ELSE': [(63, 2), (86, 2)], 'WHILE': [(43, 50)], 'DO': [], 'SWITCH': [], 'FUNCTION': [(0, 20), (20, 79)]}
